---
# Part 1 - Goal: Test if the preformance of the models (MAC) will generally imrpove due to the ensemble 
---
- preformed on the val set, so see if ensembles inproves preformance in a derirable way (acceptable recall)


In [6]:

import copy
import numpy as np
import pandas as pd
import pickle
import random
import numpy as np
import os
import itertools
from joblib import Parallel, delayed , parallel_backend
from collections import defaultdict
import math
import torch.nn as nn



from Equations_Ensembles_Dist import (

    evaluate_binary_0_1_selective_ensemble , distribution_discovery , avg_ensemble , selective_ensemble_all_agree
    , selective_ensemble_threshold_agreement , selective_ensemble_majority_vote

)


In [2]:
import pickle

with open("pair_params_maps_mac_T.pkl", "rb") as f:
    loaded_pair_maps = pickle.load(f)

# print(loaded_pair_maps["pair_params_map_2_neg01_00"].values())


FileNotFoundError: [Errno 2] No such file or directory: 'pair_params_maps_mac_T.pkl'

In [9]:
#                       SELECTIVE ENSEMBLE ALL AGREE


# all_pair_maps = {
#     "pair_params_map_2_neg01_00": pair_params_map_2_neg01_00, DONE
#     "pair_params_map_2_00_02": pair_params_map_2_00_02, DONE
#     "pair_params_map_2_02_04": pair_params_map_2_02_04,DONE
#     "pair_params_map_2_04_07": pair_params_map_2_04_07,DONE
#     "pair_params_map_3_00_02": pair_params_map_3_00_02, DONE
#     "pair_params_map_3_02_04": pair_params_map_3_02_04,DONE
#     "pair_params_map_4_00_02": pair_params_map_4_00_02 DONE
# }

pair_variables_list = [
    'pair_params_map_2_neg01_00',
    'pair_params_map_2_00_02',
    'pair_params_map_2_02_04',
    'pair_params_map_2_04_07',
    'pair_params_map_3_00_02',
    'pair_params_map_3_02_04',
    'pair_params_map_4_00_02'
]

res_all = []

for res_str in pair_variables_list:

    with parallel_backend("loky", n_jobs=-1):
        results = Parallel()(
            delayed(selective_ensemble_all_agree)(combo_list = combo_list , num_cv_sets = 4 , total_offset = 0  , INDEX = idx , combo_numbers  = combo_nums)
            for idx ,(combo_nums, combo_list) in enumerate(loaded_pair_maps[res_str].items())
        )

        res_all.append(results)



# def selective_ensemble_all_agree(combo_list , num_cv_sets , total_offset , INDEX , combo_numbers) : 




--- Running Combo 1 ---Parameters: {'learning_rate': 0.05, 'num_epochs': 150, 'batch_size': 50, 'use_bidirectional': False, 'lag': 3, 'input_size': 12, 'hidden_size': 55, 'num_layers': 1, 'use_monthly_dfs_only': True, 'use_binary_0_1': False, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': False, 'is_deterministic': True, 'seed_num': 42, 'use_monthly_predictor': False, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_monthly_daily_end_mo_predictor': False, 'use_monthly_weekly_end_mo_predictor': True, 'use_binary_0_1_custom_neg': True, 'use_binary_0_1_custom_pos': False, 'binary_0_1_cutoff_ret_rate_percentage': 0.05, 'end_value_train_set_fraction': 0.95, 'val_set_fraction': 0.05, 'num_folds': 4, 'POS_weight_multiplier': 1, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None}

--- Running Combo 1 ---Parameters: {'learning_rate': 0.005, 'num_epochs': 150

In [10]:
# import json

# file_json_metrics = f"/Users/cs/Desktop/results_MAC_Vset_pair_params_map_2_neg01_00.json"
# # file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

# with open(file_json_metrics, "w") as f_json:
#     json.dump(results, f_json, indent=2)



import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

# Your existing code
file_json_metrics = "/Users/cs/Desktop/results_MAC_Tset_pair_params_ALL.json"

with open(file_json_metrics, "w") as f_json:
    json.dump(res_all, f_json, cls=NumpyEncoder, indent=2)


---
# Part 2 - Goal: Test if the preformance of the models (GC) will generally imrpove due to the ensemble 
---
- params with various correlations are are grouped together, corelation is mearused on the VAL SET to avoid leakage! 
- the models are tested on the Test set to see if there is an improvement in preformance
- notice that this test is different from the one above , not jsut bc it is a diff machine  


In [1]:
import pickle

with open("pair_params_maps_dist_discovery_H_L_MAC_T_folds.pkl", "rb") as f:
    loaded_pair_maps = pickle.load(f)

In [4]:
#                       SELECTIVE ENSEMBLE ALL AGREE


# all_pair_maps = {
#     "pair_params_map_2_neg01_00": pair_params_map_2_neg01_00, DONE
#     "pair_params_map_2_00_02": pair_params_map_2_00_02, DONE
#     "pair_params_map_2_02_04": pair_params_map_2_02_04,DONE
#     "pair_params_map_2_04_07": pair_params_map_2_04_07,DONE
#     "pair_params_map_3_00_02": pair_params_map_3_00_02, DONE
#     "pair_params_map_3_02_04": pair_params_map_3_02_04,DONE
#     "pair_params_map_4_00_02": pair_params_map_4_00_02 DONE
# }

pair_variables_list = [
    'pair_params_map_2_neg03_01' ,     # Changed
    'pair_params_map_2_01_03',
    'pair_params_map_2_03_05',
    # 'pair_params_map_2_04_07': pair_params_map_2_04_07,          # Commented out
    'pair_params_map_3_neg03_01',     # Change
    'pair_params_map_3_01_03'    ,        # Changed
    'pair_params_map_3_03_05'     ,       # Changed
    # 'pair_params_map_3_04_07': pair_params_map_3_04_07,          # Commented out
    # 'pair_params_map_4_02_04': pair_params_map_4_02_04,          # Commented out
    # 'pair_params_map_4_04_07': pair_params_map_4_04_07           # Commented out
]


res_all = []

for res_str in pair_variables_list:

    with parallel_backend("loky", n_jobs=-1):
        results = Parallel()(
            delayed(selective_ensemble_all_agree)(combo_list = combo_list , num_cv_sets = 4 , total_offset = 0  , INDEX = idx , combo_numbers  = combo_nums)
            for idx ,(combo_nums, combo_list) in enumerate(loaded_pair_maps[res_str].items())
        )

        res_all.append(results)



# def selective_ensemble_all_agree(combo_list , num_cv_sets , total_offset , INDEX , combo_numbers) : 




--- Running Combo 1 ---Parameters: {'learning_rate': 0.05, 'num_epochs': 150, 'batch_size': 50, 'use_bidirectional': False, 'lag': 3, 'input_size': 12, 'hidden_size': 55, 'num_layers': 1, 'use_monthly_dfs_only': True, 'use_binary_0_1': False, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': False, 'is_deterministic': True, 'seed_num': 5528, 'use_monthly_predictor': False, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_monthly_daily_end_mo_predictor': False, 'use_monthly_weekly_end_mo_predictor': True, 'use_binary_0_1_custom_neg': True, 'use_binary_0_1_custom_pos': False, 'binary_0_1_cutoff_ret_rate_percentage': 0.05, 'end_value_train_set_fraction': 0.95, 'val_set_fraction': 0.05, 'num_folds': 4, 'POS_weight_multiplier': 0.7, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None}

--- Running Combo 1 ---Parameters: {'learning_rate': 0.05, 'num_epochs': 

In [5]:
# import json

# file_json_metrics = f"/Users/cs/Desktop/results_MAC_Vset_pair_params_map_2_neg01_00.json"
# # file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

# with open(file_json_metrics, "w") as f_json:
#     json.dump(results, f_json, indent=2)



import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

# Your existing code
file_json_metrics = "/Users/cs/Desktop/ensemble_pair_params_maps_dist_discovery_H_L_MAC_T_folds_ALL.json"

with open(file_json_metrics, "w") as f_json:
    json.dump(res_all, f_json, cls=NumpyEncoder, indent=2)


---
# Part 3 - Goal: Test if the preformance of the models (GC) will imrpove due to the selective model choice via distribution discovery  
---
- params with various correlations are are grouped together, corelation is mearused on the VAL SET to avoid leakage! 
- the models are tested on the Test set to see if there is an improvement in preformance
- notice that this test is different from the one above , not jsut bc it is a diff machine  


In [ ]:
import pickle

with open("pair_params_maps_gc_T.pkl", "rb") as f:
    loaded_pair_maps = pickle.load(f)


    #                       SELECTIVE ENSEMBLE ALL AGREE


# all_pair_maps = {
#     'pair_params_map_2_neg01_00' :pair_params_map_2_neg01_00 ,
#     'pair_params_map_2_00_02' : pair_params_map_2_00_02,
#     'pair_params_map_2_02_04' : pair_params_map_2_02_04 ,
#     'pair_params_map_2_04_07' : pair_params_map_2_04_07 ,
#     'pair_params_map_3_00_02' : pair_params_map_3_00_02,
#     'pair_params_map_3_02_04' : pair_params_map_3_02_04,
#     'pair_params_map_3_04_07' : pair_params_map_3_04_07,
#     'pair_params_map_4_02_04' : pair_params_map_4_02_04,
#     'pair_params_map_4_04_07' : pair_params_map_4_04_07
# }



with parallel_backend("loky", n_jobs=-1):
    results = Parallel()(
        delayed(selective_ensemble_all_agree)(combo_list = combo_list , num_cv_sets = 4 , total_offset = 0  , INDEX = idx , combo_numbers  = combo_nums)
        for idx ,(combo_nums, combo_list) in enumerate(loaded_pair_maps["pair_params_map_2_neg01_00"].items())
    )




import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

# Your existing code
file_json_metrics = "/Users/cs/Desktop/results_GC_Tset_pair_params_map_2_neg01_00.json"

with open(file_json_metrics, "w") as f_json:
    json.dump(results, f_json, cls=NumpyEncoder, indent=2)


# def selective_ensemble_all_agree(combo_list , num_cv_sets , total_offset , INDEX , combo_numbers) : 




In [ ]:
def selective_ensemble_all_agree(existing_data: list ,combo_list , num_cv_sets , total_offset , INDEX , combo_numbers , 
                                 use_existing_data : bool) : 


    if not use_existing_data:

        with parallel_backend("loky", n_jobs=1):
            all_results  = Parallel()(
                delayed(run_combo)(i, combo, 0, use_print_acc_vs_pred=False)
                for i, combo in enumerate(combo_list)
            )

        results, weights  = zip(*all_results)

    if use_existing_data:
        results = existing_data

    all_model_preds = [res["all_preds"] for res in results]  # shape: (num_models, num_folds)


    transposed_preds = list(zip(*all_model_preds))  
    stacked = [np.stack(i) for i in transposed_preds]
    stacked_bool_int = [(array > 0.5).astype(int) for array in stacked]
    predictions_folds = []

    for fold in stacked_bool_int:
        
        agreement = (fold == fold[0]).all(axis=0)  
        agreed_vals = fold[0]  
    
        result = [
            agreed_vals[i] if agreement[i] else "no agreement"
            for i in range(fold.shape[1])
        ]
        
        predictions_folds.append(result)

    actuals_folds = results[0]["all_actuals"]
    raw_actuals_folds = results[0]["raw_actuals"]


    cv_data = {}  

    for set_idx , (pred_fold,act_fold) in enumerate(zip(predictions_folds , actuals_folds )):
        
        metrics = evaluate_binary_0_1_selective_ensemble(pred_fold, act_fold , do_print=False)
        
        cv_data[f"set_{set_idx + 1}"] = metrics       
        



